In [478]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
import re, sys, random, glob, json, unidecode, unicodedata
import ipdb, time, sys, os, pickle
from IPython.core.display import display, HTML
from IPython.display import clear_output
from urllib.request import urlopen
from collections import Counter
from bs4 import BeautifulSoup  

display(HTML("<style>.container { width:100% !important; }</style>"))

# Matplotlib Configuration
pylab.rcParams.update({'legend.fontsize': 'medium',
                       'font.size'      : 12,
                       'figure.figsize' : (10, 10),
                       'axes.labelsize' : 'medium',
                       'axes.titlesize' : 'medium',
                       'axes.grid'      : 'on',
                       'xtick.labelsize': 'medium',
                       'ytick.labelsize': 'medium'})

def print_runtime(start: float, printer: bool = True) -> None:
    end = time.time()
    if printer:
        print(f'Runtime: {int((end - start) // 60)} min {int((end - start) % 60):2d} sec')
    else:
        return f'({int((end - start) // 60)} min {int((end - start) % 60):2d} sec)'

/var/folders/jf/qykqz6ks0vs56cb_zyf_7x_00000gn/T/ipykernel_1248/597161407.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


---
# Extract names from emails

In [655]:
def compute_sequence_prob(response, alpha=0.7):
    logprob = []
    for item in response.choices[0].logprobs.content:
        #print(item.token, (item.logprob))
        logprob.append(item.logprob)

    avg_logprob = sum(logprob) / len(logprob) ** .7
    return np.exp(avg_logprob)

def get_prompt(first_name, email, bio):
    prompt = f"""
    Given Name: {first_name}
    Email: {email}
    Bio: {bio}
 """
    return prompt


In [656]:
fname = 'First Name Change - Interexo 1_29_24.xlsx - Data - No Change.csv'
df = pd.read_csv(fname)
df.insert(1, 'likelihood', [0]*len(df))
df.rename({'Given Name': 'Preferred Name'}, axis=1, inplace=True)
df = df.astype({'Preferred Name':str, 'likelihood':float})
df.head()

,Preferred Name,likelihood,First Name,Last Name,Email,Company Name,Bio,Address,Country,State,City,Zip Code,Phone #,Phone #.1,Website,Profile Urls
0,nan,0.0,Christopher,Cheung,chris.cheung@fbxbrokers.com,Florida Business Exchange,Chris Cheung is a \t\t\t\t\t\tTampa business b...,550 Reo St.\t\t\t\t\t\t\t\t\t\t\t\t\tSuite 300,United States,FL,Tampa,33602,813-785-9089,NaN,https://fbxbrokers.com,https://brokers.interexo.com/united-states_flo...
1,nan,0.0,William,Gaines,bill@franpro.com,Franprocom,William Gaines is a business broker of the we...,Bowie,United States,MD,20715,NaN,443-583-3070,NaN,https://franpro.com,https://brokers.interexo.com/united-states_mar...
2,nan,0.0,Anthony,Shard,tony@shardmb.com,Shard Marketing,Anthony Shard is a \t\t\t\t\t\tCortez business...,12325 Baypointe Ter,United States,FL,Cortez,34215-2566,810-577-5291,NaN,https://shardmb.com,https://brokers.interexo.com/united-states_flo...
3,nan,0.0,Antonio,Bardy,tonybardy@gmail.com,Florida Connexion Properties,Antonio Bardy is a \t\t\t\t\t\tWindermere busi...,1961 Westhill Run,United States,FL,Windermere,34786-6230,407-535-3639,NaN,NaN,https://brokers.interexo.com/united-states_flo...
4,nan,0.0,Elizabeth,Berger,liz@lizberger.com,Berger Realty Group,Elizabeth Berger is a Coral Springs business b...,1955 University Dr,United States,FL,Coral Springs,33071,954-344-8222,954-650-7848,https://lizberger.com/,https://brokers.interexo.com/united-states_flo...


In [747]:
from openai import OpenAI
client = OpenAI()

start = time.time()
model = "gpt-3.5-turbo"
# model = "gpt-4-0125-preview"

sys_message = f"""Given the values for "Given Name", "Email" and "Bio" of a company employee, your job is to predict what his/her Preferred Name is. For instance, a person with the given name "Anthony" might prefer being called "Tony", and the evidence of it could be that his email is "tony_pearson@gmail.com" or in his Bio he might be referred to as Tony instead of Anthony. However, if you have no evidence for his "Preferred Name" being different than his "Given Name", then output the "Given Name".
Provide your response as a single string and nothing else. Do not include any other text, or any explanation or any special formatting like using backticks (``). and Don't make up a full sentence.

I'm providing the "Given Name", "Email", and the "Bio" of a business professional below."""

p = 1000
for idx, row in df[p:p+1].iterrows():
    print(f"{idx:3d}   given_name:{row['First Name']}", end="")
    
    prompt = get_prompt(row['First Name'], row['Email'], row['Bio'])
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": sys_message},
            {"role": "user", "content": prompt}
        ],
        top_p=0.00,
        logprobs=True,
        top_logprobs=5
    )
    
    content = response.choices[0].message.dict()['content']
    prob = compute_sequence_prob(response)
    df.loc[idx, 'Preferred Name'] = content
    df.loc[idx, 'likelihood'] = round(prob, 2)
    print(f', preferred_name:{content+", ":8s} prob:{round(prob,2)}')
    
    compute_sequence_prob(response)


1000   given_name:Don, preferred_name:Don,     prob:1.0


In [748]:

compute_sequence_prob(response)

0.9999597547668612

In [749]:
for item in response.choices[0].logprobs.content[0].top_logprobs:
    print(item, np.exp(item.logprob))

TopLogprob(token='Don', bytes=[68, 111, 110], logprob=-4.0246043e-05) 0.9999597547668612
TopLogprob(token=' Don', bytes=[32, 68, 111, 110], logprob=-11.846285) 7.165125364838587e-06
TopLogprob(token='The', bytes=[84, 104, 101], logprob=-12.266006) 4.709136500362086e-06
TopLogprob(token='Dan', bytes=[68, 97, 110], logprob=-12.2915) 4.590599189669888e-06
TopLogprob(token='Donald', bytes=[68, 111, 110, 97, 108, 100], logprob=-12.2979965) 4.560873024598006e-06
